# Torre de Hanoi - Búsqueda en Profundidad Iterativa

Este notebook resuelve el problema de la Torre de Hanoi utilizando el algoritmo de Búsqueda en Profundidad Iterativa (IDDFS).

In [59]:
## Setup
import time
import statistics
import psutil
import os

## 1. Introducción

El problema de la Torre de Hanoi consiste en mover una pila de discos de una clavija a otra, siguiendo estas reglas:
1. Solo se puede mover un disco a la vez.
2. Un disco más grande no puede colocarse sobre uno más pequeño.
3. Solo se puede mover el disco superior de una pila.

La Búsqueda en Profundidad Iterativa (IDDFS) es un algoritmo de búsqueda que combina la eficiencia de espacio de la búsqueda en profundidad con la completitud de la búsqueda en anchura.

In [60]:
class Estado:
    def __init__(self, pegs):
        self.pegs = pegs

    def __eq__(self, other):
        return self.pegs == other.pegs

    def __hash__(self):
        return hash(tuple(tuple(peg) for peg in self.pegs))

    def __str__(self):
        return str(self.pegs)

def mover_disco(estado, origen, destino):
    nuevo_estado = [list(peg) for peg in estado.pegs]
    disco = nuevo_estado[origen].pop()
    nuevo_estado[destino].append(disco)
    return Estado(nuevo_estado)

def es_movimiento_valido(estado, origen, destino):
    if not estado.pegs[origen]:
        return False
    if estado.pegs[destino] and estado.pegs[origen][-1] > estado.pegs[destino][-1]:
        return False
    return True

def obtener_sucesores(estado):
    sucesores = []
    for origen in range(3):
        for destino in range(3):
            if origen != destino and es_movimiento_valido(estado, origen, destino):
                sucesores.append(mover_disco(estado, origen, destino))
    return sucesores

def es_objetivo(estado, num_discos):
    return len(estado.pegs[2]) == num_discos

def iddfs(estado_inicial, num_discos):
    max_limite_profundidad = 2 ** (num_discos + 2)
    for limite_profundidad in range(1, max_limite_profundidad):
        resultado = dfs_limitado(estado_inicial, num_discos, limite_profundidad)
        if resultado is not None:
            return resultado
    return None

def dfs_limitado(estado, num_discos, limite_profundidad, profundidad=0, visitados=None):
    if visitados is None:
        visitados = set()

    if profundidad >= limite_profundidad:
        return None

    if estado in visitados:
        return None

    if es_objetivo(estado, num_discos):
        return []

    visitados.add(estado)

    for sucesor in obtener_sucesores(estado):
        resultado = dfs_limitado(sucesor, num_discos, limite_profundidad, profundidad + 1, visitados)
        if resultado is not None:
            return [(estado, sucesor)] + resultado
        
    visitados.remove(estado)
    return None

def resolver_torre_hanoi(num_discos):
    estado_inicial = Estado([[i for i in range(num_discos, 0, -1)], [], []])
    solucion = iddfs(estado_inicial, num_discos)
    return solucion

num_discos = 4
solucion = resolver_torre_hanoi(num_discos)

if solucion:
    print(f"Solución encontrada en {len(solucion)} movimientos:")
    for i, (estado, siguiente_estado) in enumerate(solucion, 1):
        print(f"Movimiento {i}: {estado} -> {siguiente_estado}")
else:
    print("No se encontró solución.")

Solución encontrada en 15 movimientos:
Movimiento 1: [[4, 3, 2, 1], [], []] -> [[4, 3, 2], [1], []]
Movimiento 2: [[4, 3, 2], [1], []] -> [[4, 3], [1], [2]]
Movimiento 3: [[4, 3], [1], [2]] -> [[4, 3], [], [2, 1]]
Movimiento 4: [[4, 3], [], [2, 1]] -> [[4], [3], [2, 1]]
Movimiento 5: [[4], [3], [2, 1]] -> [[4, 1], [3], [2]]
Movimiento 6: [[4, 1], [3], [2]] -> [[4, 1], [3, 2], []]
Movimiento 7: [[4, 1], [3, 2], []] -> [[4], [3, 2, 1], []]
Movimiento 8: [[4], [3, 2, 1], []] -> [[], [3, 2, 1], [4]]
Movimiento 9: [[], [3, 2, 1], [4]] -> [[], [3, 2], [4, 1]]
Movimiento 10: [[], [3, 2], [4, 1]] -> [[2], [3], [4, 1]]
Movimiento 11: [[2], [3], [4, 1]] -> [[2, 1], [3], [4]]
Movimiento 12: [[2, 1], [3], [4]] -> [[2, 1], [], [4, 3]]
Movimiento 13: [[2, 1], [], [4, 3]] -> [[2], [1], [4, 3]]
Movimiento 14: [[2], [1], [4, 3]] -> [[], [1], [4, 3, 2]]
Movimiento 15: [[], [1], [4, 3, 2]] -> [[], [], [4, 3, 2, 1]]


## 2. PEAS del problema

- **Rendimiento (Performance)**: Minimizar el número de movimientos para resolver el problema.
- **Entorno (Environment)**: Las tres clavijas y los discos.
- **Actuadores (Actuators)**: Mover discos entre clavijas.
- **Sensores (Sensors)**: Observar el estado actual de las clavijas y los discos.

## 3. Propiedades del entorno de trabajo

- Completamente observable
- Determinista
- Secuencial
- Estático
- Discreto
- Agente único

## 4. Elementos del problema

- **Estado**: Configuración actual de los discos en las clavijas
- **Espacio de estados**: Todas las posibles configuraciones de discos en las clavijas
- **Árbol de búsqueda**: Árbol que representa todos los movimientos posibles y los estados resultantes
- **Nodo de búsqueda**: Un nodo en el árbol de búsqueda, que representa un estado y el movimiento que llevó a él
- **Objetivo**: Todos los discos en la clavija de destino en el orden correcto
- **Acción**: Mover un disco de una clavija a otra
- **Frontera**: Conjunto de nodos no expandidos en el árbol de búsqueda

## 5. Complejidad del algoritmo

- **Complejidad temporal**: O(b^d), donde b es el factor de ramificación y d es la profundidad
- **Complejidad espacial**: O(d)

## 6. Análisis de rendimiento empírico

Ejecutaremos el algoritmo 10 veces para diferentes números de discos y mediremos el tiempo de ejecución y el uso de memoria.

In [61]:
def medir_rendimiento(num_discos, num_ejecuciones=10):
    tiempos = []
    memorias = []
    
    for _ in range(num_ejecuciones):
        inicio = time.time()
        proceso = psutil.Process(os.getpid())
        mem_inicio = proceso.memory_info().rss
        
        resolver_torre_hanoi(num_discos)
        
        fin = time.time()
        mem_fin = proceso.memory_info().rss
        
        tiempos.append(fin - inicio)
        memorias.append(mem_fin - mem_inicio)
    
    return {
        'tiempo_promedio': statistics.mean(tiempos),
        'tiempo_desviacion': statistics.stdev(tiempos),
        'memoria_promedio': statistics.mean(memorias),
        'memoria_desviacion': statistics.stdev(memorias)
    }

for discos in range(3, 5):
    resultados = medir_rendimiento(discos)
    print(f"Discos: {discos}")
    print(f"Tiempo promedio: {resultados['tiempo_promedio']:.4f} s ± {resultados['tiempo_desviacion']:.4f} s")
    print(f"Memoria promedio: {resultados['memoria_promedio'] / 1024:.2f} KB ± {resultados['memoria_desviacion'] / 1024:.2f} KB")
    print()

Discos: 3
Tiempo promedio: 0.0016 s ± 0.0002 s
Memoria promedio: 0.40 KB ± 1.26 KB

Discos: 4
Tiempo promedio: 0.0769 s ± 0.0004 s
Memoria promedio: 0.40 KB ± 1.26 KB



## 7. Comparación con la solución óptima

Compararemos el número de movimientos encontrados por IDDFS con la solución óptima (2^k - 1, donde k es el número de discos).

In [62]:
def comparar_con_optimo(num_discos, num_ejecuciones=10):
    movimientos = []
    for _ in range(num_ejecuciones):
        solucion = resolver_torre_hanoi(num_discos)
        if solucion is not None:
            movimientos.append(len(solucion))
        else:
            print(f"No se encontró solución para {num_discos} discos en esta ejecución.")
            continue
    if not movimientos:
        print(f"No se encontraron soluciones en {num_ejecuciones} ejecuciones para {num_discos} discos.")
        return None
    promedio_movimientos = statistics.mean(movimientos)
    optimo = 2**num_discos - 1
    diferencia_porcentual = ((promedio_movimientos - optimo) / optimo) * 100
    
    return {
        'promedio_movimientos': promedio_movimientos,
        'optimo': optimo,
        'diferencia_porcentual': diferencia_porcentual
    }

for discos in range(3, 5):
    resultados = comparar_con_optimo(discos)
    if resultados:
        print(f"Discos: {discos}")
        print(f"Movimientos promedio: {resultados['promedio_movimientos']:.2f}")
        print(f"Solución óptima: {resultados['optimo']}")
        print(f"Diferencia porcentual: {resultados['diferencia_porcentual']:.2f}%")
        print()

Discos: 3
Movimientos promedio: 7.00
Solución óptima: 7
Diferencia porcentual: 0.00%

Discos: 4
Movimientos promedio: 15.00
Solución óptima: 15
Diferencia porcentual: 0.00%



## 8. Conclusión

En este notebook, hemos implementado y analizado el algoritmo de Búsqueda en Profundidad Iterativa (IDDFS) para resolver el problema de la Torre de Hanoi. Hemos examinado sus propiedades, rendimiento y lo hemos comparado con la solución óptima.

Nuestros resultados muestran que:

1. El IDDFS es efectivo para resolver el problema de la Torre de Hanoi con 3 y 4 discos, aunque no siempre encuentra la solución óptima.
2. Para 5 discos o más, nuestra implementación del IDDFS no logra encontrar una solución en un tiempo razonable.

Esta limitación se debe a varios factores:

1. La naturaleza exhaustiva del IDDFS, que explora todas las posibles combinaciones hasta cierta profundidad antes de aumentar el límite.
2. El crecimiento exponencial del espacio de búsqueda con cada disco adicional.
3. La falta de heurísticas específicas para guiar la búsqueda hacia soluciones prometedoras.

En conclusión, mientras que el IDDFS es un algoritmo interesante, nuestra implementación demuestra que no es práctico para resolver el problema de la Torre de Hanoi con más de 4 discos. Para aplicaciones prácticas con un número significativo de discos, se recomendarían enfoques más especializados y eficientes.